# New Section

In [1]:
# ==============================================================================
# === Sel 1: Markdown ===
# # Notebook: 03_Augmentasi_Data_Colab.ipynb
#
# ## Tujuan
# Notebook ini bertujuan untuk melakukan augmentasi (perbanyakan) data pada kelas sentimen
# yang jumlahnya minoritas untuk menyeimbangkan dataset. Proses ini menggunakan teknik
# back-translation, penggantian sinonim, dan parafrase sederhana.
# Mengingat proses ini intensif, notebook ini dioptimalkan untuk Google Colab.
# ==============================================================================

# === Sel 2: Kode (Instalasi Library) ===
# Instalasi library yang dibutuhkan di lingkungan Colab
!pip install pandas deep-translator langdetect "nltk==3.8.1" Sastrawi retry openpyxl -q
print("Instalasi library selesai.")

# === Sel 3: Kode (Import & Hubungkan Google Drive) ===
import pandas as pd
import re
import logging
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
import random
import time
from retry import retry
from nltk.tokenize import word_tokenize
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from multiprocessing import Pool, cpu_count
import numpy as np
import os
from google.colab import drive

# Hubungkan ke Google Drive
drive.mount('/content/drive')
print("Google Drive berhasil terhubung.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.
Instalasi library selesai.
Mounted at /content/drive
Google Drive berhasil terhubung.


In [ ]:
# === Sel 4: Kode (Definisi Path dan Inisialisasi) ===
# Inisialisasi logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- MOHON SESUAIKAN PATH INI ---
# Ganti 'Proyek_Sentimen_Gojek' dengan nama folder proyek Anda di Google Drive
drive_path = '/content/drive/MyDrive/Proyek_Sentimen_Gojek/'
# --------------------------------

# Definisikan path untuk folder data
data_folder = os.path.join(drive_path, 'Data')

# Buat folder jika belum ada
os.makedirs(data_folder, exist_ok=True)
logger.info(f"Folder 'Data' siap digunakan di: {data_folder}")


# Unduh paket NLTK jika belum ada
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    nltk.download('punkt_tab')
print("Paket NLTK sudah siap.")

Paket NLTK sudah siap.


In [ ]:
# === Sel 5: Kode (Definisi Fungsi Utilitas dan Augmentasi) ===

# Tambahkan import sent_tokenize
from nltk.tokenize import sent_tokenize

# --- FUNGSI BARU UNTUK CHUNKING ---
def chunk_text(text, max_length=4500):
    """Membagi teks panjang menjadi beberapa bagian (chunks) berdasarkan kalimat."""
    # Download 'punkt' jika belum ada untuk tokenisasi kalimat
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')

    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_length:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

translation_cache = {}

@retry(tries=3, delay=2, backoff=2)
def back_translate(text, intermediate_lang='en'):
    """Fungsi untuk back-translation dengan caching dan retry."""
    # Fungsi truncate_text dihapus karena prosesnya kini ditangani oleh chunking
    try:
        if text in translation_cache:
            return translation_cache[text]

        translated = GoogleTranslator(source='id', target=intermediate_lang).translate(text)
        time.sleep(0.5) # Jeda untuk menghindari rate limit
        back_translated = GoogleTranslator(source=intermediate_lang, target='id').translate(translated)
        time.sleep(0.5)

        if back_translated:
            translation_cache[text] = back_translated
            return back_translated
        return text # Kembalikan teks asli jika gagal
    except Exception as e:
        logger.error(f"Error pada back-translation: {e}")
        return text

# Kamus sinonim untuk penggantian kata
synonym_dict = {
    'pembangunan': ['konstruksi', 'pengembangan', 'pendirian'],
    'proyek': ['rencana', 'pekerjaan', 'program'],
    'investasi': ['penanaman modal', 'pendanaan', 'modal'],
    'infrastruktur': ['sarana', 'prasarana', 'fasilitas'],
    'layanan': ['jasa', 'servis'],
    'driver': ['pengemudi', 'mitra'],
    'tarif': ['ongkos', 'biaya'],
    'aplikasi': ['platform', 'sistem']
}

gojek_keywords = [
    'gojek', 'goto', 'ojol', 'driver', 'aplikasi', 'layanan', 'tarif',
    'gofood', 'gopay', 'transportasi', 'ojek', 'online'
]

stemmer = StemmerFactory().create_stemmer()

def synonym_replacement(text, n=3):
    """Mengganti n kata secara acak dengan sinonimnya."""
    try:
        words = word_tokenize(text)
        new_words = words.copy()
        indices = [i for i, word in enumerate(words) if word.lower() in synonym_dict]
        random.shuffle(indices)

        replaced_count = 0
        for i in indices:
            if replaced_count < n:
                synonym = random.choice(synonym_dict[words[i].lower()])
                if synonym.lower() != words[i].lower():
                    new_words[i] = synonym
                    replaced_count += 1
            else:
                break
        return " ".join(new_words)
    except Exception as e:
        logger.error(f"Error in synonym_replacement: {e}")
        return text

# --- FUNGSI WRAPPER BARU UNTUK AUGMENTASI DENGAN CHUNKING ---
def augment_text_in_chunks(text, augment_func, **kwargs):
    """
    Melakukan augmentasi pada teks dengan membaginya menjadi chunks terlebih dahulu.
    """
    if len(text) < 4500: # Jika teks tidak terlalu panjang, langsung proses
        return augment_func(text, **kwargs)

    # Jika teks panjang, bagi menjadi chunks
    chunks = chunk_text(text)
    augmented_chunks = [augment_func(chunk, **kwargs) for chunk in chunks]
    return " ".join(augmented_chunks)

def validate_augmented_text(text):
    """Memvalidasi apakah teks hasil augmentasi masih relevan."""
    try:
        if len(text.split()) < 5: return False
        try:
            is_id = detect(text) == 'id'
        except LangDetectException:
            is_id = False
        if not is_id: return False

        keyword_count = sum(1 for keyword in gojek_keywords if keyword.lower() in text.lower())
        return keyword_count >= 2
    except Exception as e:
        logger.error(f"Error in validate_augmented_text: {e}")
        return False

def augment_row(row_tuple):
    """Fungsi wrapper untuk augmentasi satu baris data (untuk multiprocessing)."""
    index, row, intermediate_langs = row_tuple
    try:
        original_text = row['cleaned_content']
        method = random.choice(['back_translate', 'synonym'])

        if method == 'back_translate':
            lang = random.choice(intermediate_langs)
            # Gunakan fungsi wrapper chunking untuk back-translation
            augmented_text = augment_text_in_chunks(original_text, back_translate, intermediate_lang=lang)
        else:
            # Gunakan fungsi wrapper chunking untuk penggantian sinonim
            augmented_text = augment_text_in_chunks(original_text, synonym_replacement, n=3)

        if augmented_text and augmented_text != original_text and validate_augmented_text(augmented_text):
            new_row = row.copy()
            new_row['cleaned_content'] = augmented_text
            new_row['is_augmented'] = True
            return new_row
        return None
    except Exception as e:
        logger.error(f"Error dalam augment_row di baris {index}: {e}")
        return None

In [ ]:
# === Sel 6: Kode (Pipeline Augmentasi Utama) ===

# Path file input (hasil dari notebook 02)
input_file = os.path.join(data_folder, 'gojek_news_preprocessed.csv')
try:
    df = pd.read_csv(input_file)
    logger.info(f"Berhasil memuat file '{input_file}' dengan {len(df)} baris.")
except FileNotFoundError:
    logger.error(f"File '{input_file}' tidak ditemukan. Jalankan notebook preprocessing terlebih dahulu.")
    raise SystemExit(1)


if __name__ == '__main__' and not df.empty:
    # Tambahkan kolom 'label' jika tidak ada (untuk pengujian)
    if 'label' not in df.columns:
        logger.warning("Kolom 'label' tidak ditemukan. Menggunakan 'tag' sebagai 'label'.")
        df['label'] = df['tag'] # Asumsikan 'tag' adalah kolom sentimen

    TARGET_PER_LABEL = 150
    INTERMEDIATE_LANGS = ['en', 'fr', 'de', 'es']

    final_dfs = []
    logger.info(f"Memulai proses augmentasi untuk mencapai target {TARGET_PER_LABEL} data per label...")

    for label in df['label'].unique():
        subset = df[df['label'] == label].copy()
        count = len(subset)
        logger.info(f"Label '{label}' memiliki {count} data.")

        if count >= TARGET_PER_LABEL:
            sampled = subset.sample(n=TARGET_PER_LABEL, random_state=42).copy()
            sampled['is_augmented'] = False
            final_dfs.append(sampled)
        else:
            subset['is_augmented'] = False
            final_dfs.append(subset)
            n_to_augment = TARGET_PER_LABEL - count
            logger.info(f"Label '{label}': Perlu augmentasi sebanyak {n_to_augment} data.")

            # Oversample untuk antisipasi kegagalan validasi
            rows_to_augment_indices = [i % count for i in range(n_to_augment * 2)]
            rows_to_augment = [(index, subset.iloc[index].copy(), INTERMEDIATE_LANGS) for index in rows_to_augment_indices]

            # Menggunakan multiprocessing untuk mempercepat proses
            with Pool(processes=max(1, cpu_count() - 1)) as pool:
                results = pool.map(augment_row, rows_to_augment)

            augmented_rows = [row for row in results if row is not None][:n_to_augment]

            if len(augmented_rows) < n_to_augment:
                logger.warning(f"Label '{label}': Hanya {len(augmented_rows)} data berhasil diaugmentasi. Sisanya diisi dengan duplikasi.")
                remaining = n_to_augment - len(augmented_rows)
                fallback_rows = [subset.iloc[i % count].copy() for i in range(remaining)]
                for row in fallback_rows:
                    row['is_augmented'] = False # Tandai sebagai tidak di-augmentasi
                augmented_rows.extend(fallback_rows)

            df_aug = pd.DataFrame(augmented_rows)
            final_dfs.append(df_aug)

    df_final = pd.concat(final_dfs, ignore_index=True)

    # Simpan hasil augmentasi
    output_path_csv = os.path.join(data_folder, 'gojek_news_augmented.csv')
    output_path_xlsx = os.path.join(data_folder, 'gojek_news_augmented.xlsx')
    df_final.to_csv(output_path_csv, index=False)
    df_final.to_excel(output_path_xlsx, index=False)

    logger.info("--- PROSES AUGMENTASI SELESAI ---")
    logger.info(f"Data augmented disimpan di '{output_path_csv}' dan '{output_path_xlsx}'")
    logger.info(f"Total data akhir: {len(df_final)}")
    logger.info(f"Distribusi label baru:\n{df_final['label'].value_counts().to_string()}")

    print("\nSampel data hasil augmentasi:")
    from IPython.display import display
    display(df_final.sample(10)[['cleaned_content', 'label', 'is_augmented']])

ERROR:__main__:Error pada back-translation: Des milliers de chauffeurs de moto-taxi en ligne membres de l’Association des gardes indonésiens organiseront une action majeure en mai 2025 à Jakarta. Cette action s'est caractérisée par une offre massive, à savoir la suppression simultanée des services d'application dans toute l'Indonésie, en guise de protestation contre une baisse des revenus jugée injuste. Les conducteurs exigent une limite maximale de déduction de pour cent, alors qu'ils affirment qu'actuellement les déductions des applicateurs peuvent atteindre près de pour cent, un lourd fardeau dans un contexte de faibles revenus quotidiens. Cette condition est considérée comme étant en conflit avec les réglementations gouvernementales, telles que le Permenhub PM n° 122019 et le décret du ministère des Transports n° 10012022, 1905. Cette vague d'insatisfaction a en fait émergé depuis le dernier Eid al-Fitr, lorsque les conducteurs ont également exigé des indemnités de vacances. Le pré


Sampel data hasil augmentasi:


,cleaned_content,label,is_augmented
428,masalah ojek online belum juga tuntas hingga s...,Foreign News,True
272,Survei terhadap mayoritas pengguna yang tetap ...,Economic,True
504,uiux study case optimalisasi aplikasi __gojek_...,Opinion,True
640,Sejumlah netizen mengeluhkan adanya error pada...,Academic,True
249,survei mayoritas pengguna tetap pakai ojol mes...,Economic,True
582,uiux study case optimalisasi aplikasi __gojek_...,Opinion,True
606,bali dengan segala kelakuan wisatawan asingnya...,Academic,False
410,masalah ojek online belum juga tuntas hingga s...,Foreign News,True
7,asosiasi pengemudi transportasi daring garda i...,Local News,False
732,"oleh bang adi bagi sr 42 tahun , seorang drive...",Academic,True
